In [1]:
import tensorflow as tf
import glob
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
# specify whether we want to create tfrecords for test or training data
# NOTE: this has to be run with 'test' and 'train'
test_or_train = 'test'

# the paths to the images
face_image_path = './data/face_images/' + test_or_train + '/face/'
none_image_path = './data/face_images/' + test_or_train + '/none/'

# get a list of all files
# NOTE: depending on the file exension, may have to use '.jpeg' or '.jpg'
face_image_names = glob.glob(face_image_path + '*.jpg')  # '.jpg' or '.jpeg'
none_image_names = glob.glob(none_image_path + '*.jpg')  # '.jpg' or '.jpeg'
# create labels for all of the files
# 1 for face
# 0 for non-face
face_image_labels = [1 for x in face_image_names]
none_image_labels = [0 for x in none_image_names]

# combine the files and labels into one list
all_images = face_image_names + none_image_names
all_labels = face_image_labels + none_image_labels

# specify the default batch_size
batch_size = 100

# image dimensions
image_height = 60
image_width = 80
image_channels = 3

In [3]:
# create a queue structure of all files to read
filename_queue = tf.train.string_input_producer(all_images, shuffle=True)

# a reader to use to read in data from the files in a queue
reader = tf.WholeFileReader()

In [4]:
def _int64_feature(value):
    ''' convert an integer into data appropriate to be written to tfrecord file '''
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    ''' convert byte data into data appropriate to be written to tfrecord file '''
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# define some operations here for encoding an image in a
# string format to be written to tfrecord file
raw_image_data = tf.placeholder("float", [image_height, image_width, image_channels])
raw_image_data_uint8 = tf.cast(raw_image_data, tf.uint8)
image_string_data = tf.image.encode_jpeg(raw_image_data_uint8)

# images and labels array as input
def convert_to(images, labels, name):
    ''' takes a group of images and labels and creates a tfrecord file '''
    
    # a check to see if the number of images and labels is consistent
    num_examples = labels.shape[0]
    if images.shape[0] != num_examples:
        raise ValueError("Images size %d does not match label size %d." %
                     (images.shape[0], num_examples))
    
    # create the file to be written to
    filename = os.path.join('./tfrecord_files/', name + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    
    # get one example at a time and write it to the tfrecord
    for index in range(num_examples):
        # reshape image into acceptable format
        image_data = np.reshape(images[index], [image_height, image_width, image_channels])
        
        # convert image data into string type data as required by tfrecord writers
        image_string = sess.run([image_string_data],
                                feed_dict={raw_image_data: image_data})
        
        # create one example to be written to tfrecord file
        example = tf.train.Example(features=tf.train.Features(feature={
            'label': _int64_feature(int(labels[index])),
            'image_raw': _bytes_feature(image_string[0])}))
        
        # writes the example to the tfrecord
        writer.write(example.SerializeToString())

In [5]:
# perform initialization of tensorflow variables
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)

In [ ]:
# some code to experiment with creating tfrecord files
# and displaying the images

# this code does not need to be run but might help you to understand
# the intermediate steps of what is going on

filename, content = reader.read(filename_queue)

image = tf.image.decode_jpeg(content, channels=3) 
image1 = tf.cast(image, tf.float32)
image2= tf.image.resize_images(image1, [60, 80])

test_or_train = 'prac'

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    
    images = []
    labels = []
    for i in range(5):
        file, im = sess.run([filename, image2])
        
        lab = 0
        if b'/face/' in file:
            lab = 1
        print(lab)
        
        im = np.reshape(im, [60, 80, 3])
        
        plt.imshow(255*im)
        plt.show()
        
        images.append(im)
        labels.append([all_labels[i]])
        
    images = np.array(images)
    print(images.shape)
    labels = np.array(labels)
    print(labels.shape)
    #convert_to(images, labels, 'practice')

    coord.request_stop()
    coord.join(threads)

In [6]:
# retrieves images and image file names from the queue of image file names
filename, content = reader.read(filename_queue)

# specify how to decode an image
image = tf.image.decode_jpeg(content)
image1 = tf.cast(image, tf.float32)
image2 = tf.image.resize_images(image1, [image_height, image_width])

# start using the tf session
with tf.Session() as sess:
    
    # start thread coordinators for running operations in tensorflow
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    
    # get the number of images in the directory
    num_images = len(all_images)

    print('preparing ' + str(num_images) + ' images.')
    
    # some initializations for the creation of tfrecords
    images = []
    labels = []
    batch_size = batch_size
    # keep track of which batch we are on
    batch_num = 0
    # keep track of which image in a batch we have processed
    batch_counter = 0
    
    for i in range(num_images):
        batch_counter += 1
        
        # get the image filename and string encoded image
        file, im = sess.run([filename, image2])
        
        # determine the label of the image from the filename
        lab = 0
        if b'/face/' in file:
            lab = 1
        
        images.append(im)
        labels.append(lab)
        
        # once we have enough images for our batch, we can write to the tfrecords
        # file and then start over again
        if batch_counter >= batch_size:
            images = np.array(images)
            labels = np.array(labels)
            convert_to(images, labels, test_or_train + str(batch_num))
            batch_counter = 0
            batch_num += 1
            images = []
            labels = []
    
    print('tf records created!!')

    # stop the threads and wait for the to finish
    coord.request_stop()
    coord.join(threads)



preparing 248 images.
Writing ./tfrecord_files/test0.tfrecords
Writing ./tfrecord_files/test1.tfrecords
tf records created!!
